In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews'

In [5]:
from dataclasses import dataclass

@dataclass
class DataTransformationConfig:
    root_dir: str
    data_path: str   


    


In [6]:
from sentimentanalyzer.constants import *
from sentimentanalyzer.utils.common import read_yaml, create_directories
from pathlib import Path
import yaml
from typing import Union
from typing import Union
from pathlib import Path
from sentimentanalyzer.utils.common import read_yaml, create_directories  # adjust import as needed
from sentimentanalyzer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH 

In [7]:

from typing import Union
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Union[str, Path] = CONFIG_FILE_PATH,
        params_filepath: Union[str, Path] = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        print(">>> CONFIG CONTENTS:", self.config)         # debug
        print(">>> CONFIG KEYS:", list(self.config.keys()))
        self.params = read_yaml(params_filepath)
        print(">>> PARAMS CONTENTS:", self.params)         
        print(">>> PARAMS KEYS:", list(self.params.keys()))  

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path = config.data_path
            )

        return data_transformation_config


In [8]:


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
from sentimentanalyzer.logging import logger
from sentimentanalyzer.constants import *
from pathlib import Path
import numpy as np
import pandas as pd
from typing import List
import tensorflow as tf
import re
from src.sentimentanalyzer.utils.common import load_saved_labels_and_texts,preprocess_ft_txt


In [9]:
import os
import re
from typing import List

class DataTransformation:
    def __init__(self, config):
        self.config = config
        
        # Load preprocessed data: expects a dict with 'train' and 'test' keys,
        # each is a tuple: (labels_list, texts_list)
        self.data = load_saved_labels_and_texts(self.config.data_path)
        
        if not self.data or "train" not in self.data or "test" not in self.data:
            raise ValueError("❌ Data must contain 'train' and 'test' keys with labels and texts.")
        
        train_labels, train_texts = self.data.get("train")
        test_labels, test_texts = self.data.get("test")
        
        print("📊 Train:", len(train_labels), train_texts[:2])
        print("📊 Test:", len(test_labels), test_texts[:2])
        
        # Normalize texts right away
        self.train_texts = self.normalize_texts(train_texts)
        self.test_texts = self.normalize_texts(test_texts)
        self.train_labels = train_labels
        self.test_labels = test_labels
        
        # Save normalized data back in fastText format
        self.save_to_ft_format(
            self.train_labels, self.train_texts, 
            os.path.join(self.config.root_dir, "train_ft.txt")
        )
        self.save_to_ft_format(
            self.test_labels, self.test_texts, 
            os.path.join(self.config.root_dir, "test_ft.txt")
        )
    
    def normalize_texts(self, texts: List[str]) -> List[str]:
        NON_ALPHANUM = re.compile(r'[\W]')
        NON_ASCII = re.compile(r'[^a-z0-1\s]')
        normalized_texts = []
        
        for text in texts:
            lower = text.lower()
            no_punctuation = NON_ALPHANUM.sub(' ', lower)
            no_non_ascii = NON_ASCII.sub('', no_punctuation)
            normalized_texts.append(no_non_ascii.strip())
        
        return normalized_texts
    
    def save_to_ft_format(self, labels: List[str], texts: List[str], filepath: str):
        assert len(labels) == len(texts), "❌ Labels and texts length mismatch"
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        
        with open(filepath, 'w', encoding='utf-8') as f:
            for label, text in zip(labels, texts):
                f.write(f"__label__{label} {text}\n")
        
        print(f"💾 Saved: {filepath}")



In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
except Exception as e:
    raise RuntimeError("Error occurred during data transformation.") from e


[2025-06-18 01:10:37,527: INFO: common: yaml file: config\config.yaml loaded successfully]
>>> CONFIG CONTENTS: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/Spencer0013/NLP-Text-Summarizer-Project/raw/refs/heads/main/Dataa.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_preprocessing': {'root_dir': 'artifacts/data_preprocessing', 'ingestion_dir': 'artifacts/data_ingestion', 'output_dir': 'artifacts/data_preprocessing'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_preprocessing'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformation', 'model_save_path': 'artifacts/model_trainer/sentiment_model'}}
>>> CONFIG KEYS: ['artifacts_root', 'data_ingestion', 'data_preprocessing', 'data_transformation', 'model_trainer']
[2025-06-18 01:10:37,535: INFO: commo